# 特征提取
计算单次操作中的燃弧相角、时长、能量、电量、功率（由于后期样本发生劣化，燃弧时间偏长，因此只取前期样本）
- 燃弧相角：
- 燃弧时长(arcing_duration)：$t$ 
- 燃弧能量(arcing_energy)：$\int i(t)u(t)dt$
- 燃弧电量(arcing_electricity)：$\int i(t)^2dt$
- 燃弧功率(arcing_power)：$\frac{\int i(t)u(t)dt}{t}$

In [1]:
import re
import os
import time
import csv
import math
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import myUtils

### 2.0 参数设置

In [2]:
root_data_path = '../data/'
processed_data_path = root_data_path + '1_processed_data/'
src_data_path = processed_data_path + '2_smoothed_data/'
src_machine_files =os.listdir(src_data_path)
print(src_machine_files)

['1.pkl', '10.pkl', '2.pkl', '3.pkl', '4.pkl', '5.pkl', '6.pkl', '7.pkl', '8.pkl', '9.pkl']


In [3]:
# 常数定义
CONST_PARAMS_DICT = {
    'frequency': 1 / 200, #频率
    'U_amplitude': 421 / np.sqrt(3) * np.sqrt(2), #424,
    'I_amplitude': 228,
    'U_eps': 5,
    'I_eps': 5,
    'three_phase': ['A','B','C']
}

### 2.1 数据分段

#### 拟合正弦函数

In [4]:
from scipy.optimize import leastsq
import warnings
# 根据x和参数输出sin函数值
def calSinByParams(x, params):
    A, f, theta_x, theta_y = params 
    return A * np.sin(2 * np.pi * f * x + theta_x) + theta_y

def calFittingResidual(params, x, y_real):
    return y_real - calSinByParams(x, params)

def getOptFittingParams(data):
    '''
    params_init:初始化拟合参数 
        params_init[0]: A 振幅
        params_init[1]: f 频率
        params_init[2]: theta_x 偏移
        params_init[3]: theta_y 
    '''

    # 待拟合数据
    X = np.arange(len(data))
    Y = np.array(data)
    
    # 初始化参数
    Y_sorted = sorted(Y)
    n = min(int(len(Y)/100), 10)
    A = (np.mean(Y_sorted[-n:]) - np.mean(Y_sorted[:n])) / 2
    
    params_init = [A, CONST_PARAMS_DICT['frequency'], 0, 0]
    
    # 最小二乘得到拟合最优解
    #params = [params_init['amplitude'], params_init['frequency'], params_init['theta_x'], params_init['theta_y']]
    
    with warnings.catch_warnings():
        try:
            params_opt, _ = leastsq(calFittingResidual, params_init, args = (X, Y), maxfev=5000)
        except Warning as e:
            print(e)
            params_opt = params_init
    
    return params_opt

#### 计算燃弧范围

In [5]:
def findArcingRange(U_data, I_data):
    """
    计算燃弧区间
    
    Args:
        U_data: 单次操作的电压数据
        I_data: 单次操作的电流数据
    
    Returns:
        arcing_range: arcing_range[0] = arcing_start：燃弧开始位置
                      arcing_range[1] = arcing_end：  燃弧结束位置
    """
    
    # 0. 初始化燃弧范围
    arcing_range = [-1, -1]
    
    n_points = 10
    data_len = len(U_data)
    
    # 1.确定燃弧结束位置：电流为0时表示燃弧结束（结束点后n_points对应的电流均小于电流阈值，结束点前n_points对应的电流均大于电流阈值）
    for i in range(data_len - n_points - 1, n_points, -1):
        #判断条件一、在[i - n_points,i]时刻内的电流是否大于0  
        n_Ii_larger_than_0 = 0
        for Ii in I_data[i - n_points : i]:
            if np.abs(Ii) > CONST_PARAMS_DICT['I_eps']:
                n_Ii_larger_than_0 += 1
        
        #判断条件二、在[i + 1, i + n_points]时刻内的电流是否接近0
        n_Ii_equals_0 = 0
        for Ii in I_data[i + 1: i + n_points]:
            if np.abs(Ii) < CONST_PARAMS_DICT['I_eps']:
                n_Ii_equals_0 += 1
        
        #若以上两个条件基本满足，则说明找到燃弧结束位置
        if n_Ii_larger_than_0 > n_points - 3 and n_Ii_equals_0 > n_points - 3:
            arcing_range[1] = i
            break
        
        
    # 2.确定燃弧开始位置
    
    # 方式一、根据残差确定燃弧开始位置，稳态时残差较小，而燃弧时残差较大
    # 截取部分合闸稳态数据，拟合得到稳态时的电压正弦波形
    end_loc = arcing_range[1] - int(1 / CONST_PARAMS_DICT['frequency'] * 5)
    start_loc = end_loc - int(1 / CONST_PARAMS_DICT['frequency'] * 3)
    U_part = np.array(U_data[start_loc : end_loc])
    params_U_close = getOptFittingParams(U_part)
    
    # 计算拟合的电压与原始数据的残差
    U_real = np.array(U_data[start_loc : arcing_range[1]])
    U_pred = calSinByParams(np.arange(len(U_real)), params_U_close)
    U_residual = np.abs(U_real - U_pred)
    
    # 判断在[i - n_points : i]时刻内的电压残差是否接近0
    arcing_range[0] = arcing_range[1]
    for i in range(len(U_residual) - 1, n_points - 1, -1):
        if U_residual[i] < CONST_PARAMS_DICT['U_eps']:
            n_U_residual_equals_0 = 0
            for u_residual in U_residual[i - n_points : i]:
                if u_residual < CONST_PARAMS_DICT['U_eps']:
                    n_U_residual_equals_0 += 1
             
            if n_U_residual_equals_0 > n_points -3:
                arcing_range[0] = start_loc + i
                break
            
            
    ''' 
    # 方式二、根据实际电压是否超过阈值判断
    for i in range(arcing_range[1], n_points, -1):
        if np.abs(U_data[i]) <= np.abs(params_U_close[0]):#CONST_PARAMS_DICT['U_eps']:
            # 判断条件一、在[i-n_point, i]时刻内的电压是否都小于电压阈值
            n_Ui_equals_0 = 0
            for Ui in U_data[i - n_points : i]:
                if np.abs(Ui) <= np.abs(params_U_close[0]):#< CONST_PARAMS_DICT['U_eps']:
                    n_Ui_equals_0 += 1
             
            if n_Ui_equals_0 > n_points / 2:
                arcing_range[0] = i
                break
    '''          
    return arcing_range

#### 绘制燃弧区间示意图

In [6]:
feature_path = root_data_path + '2_features/'
arcing_range_fig_path = feature_path + '2_arcing_range_fig/'
myUtils.makedirs(arcing_range_fig_path)

../data/2_features/2_arcing_range_fig/ already exist.


In [29]:
# 单相图片存储
for tmp_file in [src_machine_files[4]]:#src_machine_files:
    pkl_file = open(src_data_path + tmp_file, 'rb')
    #print(pkl_file)
    operation_data_list = pickle.load(pkl_file)
    pkl_file.close()
    print(tmp_file, len(operation_data_list))
    
    
    arcing_range_fig_folder = arcing_range_fig_path + re.match('(.*).pkl', tmp_file).group(1) + '/'
        
    for i in range(len(operation_data_list)):
        if i % 100 == 0:
            print(i)
        for phase in CONST_PARAMS_DICT['three_phase']:
            cur_phase_folder = arcing_range_fig_folder + phase + '/'
            myUtils.makedirs(cur_phase_folder)
            
            fig = plt.figure(figsize=(25,10))
            U_data = operation_data_list[i]['U' + phase]
            I_data = operation_data_list[i]['I' + phase]
            arcing_range = findArcingRange(U_data, I_data)
            print(arcing_range)
            continue
            show_data_range = np.arange(arcing_range[0] - 400, arcing_range[1] + 300)
            data_range = np.arange(arcing_range[0], arcing_range[1])
            plt.plot(U_data[show_data_range])
            plt.plot(I_data[show_data_range])
            plt.plot(U_data[data_range], 'r')
            plt.axvline(arcing_range[0], c = 'g', ls = ':')
            plt.axvline(arcing_range[1], c = 'g', ls = ':')
            plt.savefig(cur_phase_folder + 'operation_time_' + str(i) + '.png', dpi=100, bbox_inches ='tight' )
            plt.close(fig)

4.pkl 186
0
../data/2_features/2_arcing_range_fig/4/A/ already exist.
[6216, 6285]
../data/2_features/2_arcing_range_fig/4/B/ already exist.
[6220, 6285]
../data/2_features/2_arcing_range_fig/4/C/ already exist.
[6225, 6245]
../data/2_features/2_arcing_range_fig/4/A/ already exist.
[6309, 6384]
../data/2_features/2_arcing_range_fig/4/B/ already exist.
[6314, 6342]
../data/2_features/2_arcing_range_fig/4/C/ already exist.
[6318, 6384]
../data/2_features/2_arcing_range_fig/4/A/ already exist.
[6311, 6341]
../data/2_features/2_arcing_range_fig/4/B/ already exist.


KeyboardInterrupt: 

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

<Figure size 1800x720 with 0 Axes>

In [ ]:
# 三相图片存储
for tmp_file in [src_machine_files[4]]:#src_machine_files:
    pkl_file = open(src_data_path + tmp_file, 'rb')
    #print(pkl_file)
    operation_data_list = pickle.load(pkl_file)
    pkl_file.close()
    print(tmp_file, len(operation_data_list))
    
    show_data_range = np.arange(5700,6700)
    arcing_range_fig_folder = arcing_range_fig_path + re.match('(.*).pkl', tmp_file).group(1) + '/'
    cur_phase_folder = arcing_range_fig_folder + '三相/'
    myUtils.makedirs(cur_phase_folder)
    for i in range(len(operation_data_list)):
        if i % 100 == 0:
            print(i)
        
        fig = plt.figure(figsize=(30,30))
        for phase_idx, phase in enumerate(CONST_PARAMS_DICT['three_phase']):
            plt.subplot(3, 1, phase_idx + 1)
            U_data = operation_data_list[i]['U' + phase]
            I_data = operation_data_list[i]['I' + phase]
            arcing_range = findArcingRange(U_data, I_data)
            
            data_range = np.arange(arcing_range[0], arcing_range[1])
            plt.plot(U_data[show_data_range])
            plt.plot(I_data[show_data_range])
            plt.plot(U_data[data_range], 'r')
            plt.axvline(arcing_range[0], c = 'g', ls = ':')
            plt.axvline(arcing_range[1], c = 'g', ls = ':')
            plt.title(phase)
        plt.savefig(cur_phase_folder + 'operation_time_' + str(i) + '.png', dpi=100, bbox_inches ='tight' )
        plt.close(fig)
        

### 2.2 分阶段特征生成

#### 1) 燃弧特征提取

In [6]:
# 获得燃弧特征
def extractArcingFeatures(U_data, I_data):
    # 计算燃弧范围
    arcing_range = findArcingRange(U_data, I_data)
    #print('======', arcing_range, arcing_range[1] - arcing_range[0])
    # 燃弧时长
    arcing_duration = max(0, arcing_range[1] - arcing_range[0])
    if arcing_duration > 2000:
        print('duration>2000')
        arcing_duration = 0
    if arcing_range[1] == -1:
        print(arcing_range)
    
    # 燃弧能量
    U_arcing = np.array(U_data[arcing_range[0] : arcing_range[1]])
    I_arcing = np.array(I_data[arcing_range[0] : arcing_range[1]])
    arcing_energy = 0 if arcing_duration <= 0 else np.sum([np.abs(U_arcing[i] * I_arcing[i]) for i in range(arcing_duration)])
    
    # 燃弧电量
    arcing_electricity = 0 if arcing_duration <= 0 else np.sum([I_arcing[i] ** 2 for i in range(arcing_duration)])
    
    # 燃弧功率
    arcing_power = arcing_energy / (arcing_duration + 1)
    
    #print('==', arcing_range, arcing_duration, arcing_range[1] - arcing_range[0])
    # 计算燃弧相角
    phase_angle_df = extractArcingPhaseAngleFeatures(U_data, arcing_range)
    
    #print('1', arcing_range, arcing_duration, arcing_range[1] - arcing_range[0])
    
    # 计算开闸前的异常点位置
    #abnormal_points_loc = findAbnormalPoints(U_data, I_data, arcing_range[0])
    # 异常点最早出现的位置
    #abnormal_start = abnormal_points_loc[-1] if abnormal_points_loc else arcing_range[0]
    
    #open_duration = arcing_range[1] - abnormal_start
    arcing_df = pd.DataFrame([[arcing_range[0], arcing_range[1], arcing_duration, arcing_energy, arcing_electricity, arcing_power]], 
                             columns = ['arcing_start', 'arcing_end', 'arcing_duration', 'arcing_energy', 'arcing_electricity', 'arcing_power'])
    arcing_df = pd.concat([arcing_df, phase_angle_df], axis = 1)
    #print(arcing_range[0], arcing_range[1], arcing_duration, arcing_range[1] - arcing_range[0])
    return arcing_df


# 计算燃弧相角
def extractArcingPhaseAngleFeatures(U_data, arcing_range):
    """
    计算燃弧相角
    
    Args:
        U_data:电压数据
        arcing_range:燃弧范围
            arcing_range[0]:燃弧起始点
            arcing_range[1]:燃弧结束点
    
    Returns:
        phase_angle:燃弧相角
        phase_angle_range:燃弧相角的计算范围
            phase_angle_range[0]:计算起始点
            phase_angle_range[1]:计算结束点
    """
    n_points = 10
    data_len = len(U_data)
    # 燃弧区间异常判断
    if arcing_range[1]  == -1:
        plt.plot(U_data)
        plt.plot(I_data)
        return pd.DataFrame([[0, 0, -1, -1]], columns = ['phase_angle_degree', 'phase_angle_radian', 'phase_angle_start', 'phase_angle_end'])
    
    # 燃弧相角计算的起始位置：与燃弧的起始范围一致
    phase_angle_range = [arcing_range[0], arcing_range[1]]
    
    #确定燃弧相角计算的结束点：电压从零开始上升的位置
    for i in range(arcing_range[1] , data_len):
        #print(i)
        n_Ui_larger_0 = 0
        if U_data[i] < CONST_PARAMS_DICT['U_eps']:
            for Ui in U_data[i + 1: i + n_points]:
                if Ui > CONST_PARAMS_DICT['U_eps']:
                    n_Ui_larger_0 += 1
            
            if n_Ui_larger_0 >= n_points - 3:
                phase_angle_range[1] = i
                break
    
    #print('phase:',phase_angle_range)
    period = int(1/ CONST_PARAMS_DICT['frequency'])
    # 将燃弧相角限定在0-360范围内？？？
    phase_angle_range[1] = (phase_angle_range[1] - phase_angle_range[0]) % period + phase_angle_range[0]
    phase_angle = max(0, min(1 - 1 / period * (phase_angle_range[1] - phase_angle_range[0]), 1))
    phase_angle_degree = 360 * phase_angle
    phase_angle_radian = 2 * np.pi * phase_angle
    return pd.DataFrame([[phase_angle_degree, phase_angle_radian, phase_angle_range[0], phase_angle_range[1]]], columns = ['phase_angle_degree', 'phase_angle_radian', 'phase_angle_start', 'phase_angle_end'])

#### 2) 合闸稳态特征生成

In [8]:
# 提取合闸后稳态特征
def extractTouchFeatures(U_data, I_data):
    # 设置波形截取的起止位置
    end_loc = 6000 - int(1 / CONST_PARAMS_DICT['frequency'] * 2)
    start_loc = end_loc - int(1 / CONST_PARAMS_DICT['frequency'] * 5)
    
    # 获取接触时的电流、电压波形
    U_part = np.array(U_data[start_loc : end_loc])
    I_part = np.array(I_data[start_loc : end_loc])
    
    #plt.plot(U_data)
    #plt.plot(I_data)
    #plt.plot(np.arange(start_loc, end_loc), U_part)
    
    #拟合波形，得到接触电流、接触电压、接触电阻
    #params_init = [0, CONST_PARAMS_DICT['frequency'], 0, 0]
    U_params = getOptFittingParams(U_part)#, params_init)
    I_params = getOptFittingParams(I_part)#, params_init)
    
    return pd.DataFrame([[np.abs(U_params[0]), np.abs(I_params[0]), np.abs(U_params[0] / (I_params[0] + 0.01))]], columns = ['touch_U', 'touch_I', 'touch_R'])

### 2.3 非波形特征整合

In [7]:
# 提取单相特征
def extract1PhaseFeatures(U_data, I_data, operation_stage = 'total'):
    """
    提取单相特征
    
    Args:
        U_data:单次操作的单相电压数据
        I_data:单次操作的单相电流数据
        operation_stage:指定所要提取特征的操作阶段。可取值有：'total'表示提取所有阶段的特征（默认）；'arcing'表示提取燃弧阶段特征；'touch'表示提取合闸稳态阶段特征
    
    Returns:
        dataframe类型的特征数据
    """
    if operation_stage == 'arcing':
        return extractArcingFeatures(U_data, I_data)
    elif operation_stage == 'touch':
        return extractTouchFeatures(U_data, I_data)
    else:
        arcing_df = extractArcingFeatures(U_data, I_data)
        touch_df = extractTouchFeatures(U_data, I_data)
        return pd.concat([arcing_df, touch_df], axis = 1, sort = False)

# 提取三相特征
def extract3phaseFeatures(operation_data_list, operation_stage):
    machine_ftr_df = pd.DataFrame()
    for i, operation_data_list in enumerate(operation_data_list):
            
        operation_ftr_df = pd.DataFrame()
        for phase in CONST_PARAMS_DICT['three_phase']:
            U_data = operation_data_list['U' + phase]
            I_data = operation_data_list['I' + phase]
            cur_ftr_df = extract1PhaseFeatures(U_data, I_data, operation_stage)
            
            # 指定列名为某相的列名
            cur_ftr_cols = [phase + '_' + col for col in cur_ftr_df.columns.tolist()]
            cur_ftr_df.columns = cur_ftr_cols
            
            operation_ftr_df = pd.concat([operation_ftr_df, cur_ftr_df], axis = 1, sort = False)
        
        if i % 100 == 0:
            print(i)
        machine_ftr_df = machine_ftr_df.append(operation_ftr_df, ignore_index = True)
        
    return machine_ftr_df

#### 提取并保存特征

In [8]:
for tmp_file in src_machine_files[4:]:
    t_start = time.time()
    machine_id = re.match('(.*).pkl', tmp_file).group(1) 
    pkl_file = open(src_data_path + tmp_file, 'rb')
    operation_data_list = pickle.load(pkl_file)
    pkl_file.close()
    print(tmp_file, len(operation_data_list))
    
    operation_stage = 'arcing'
    ftr_path = root_data_path + '2_features/' + operation_stage + '/'
    myUtils.makedirs(ftr_path)
    machine_ftr_df = extract3phaseFeatures(operation_data_list, operation_stage)
    #print(machine_ftr_df.head())
    machine_ftr_df.to_csv(ftr_path + machine_id + '.csv', index = False, header = True)
    
    t_end = time.time()
    print('用时：', (t_end - t_start))

4.pkl 186
../data/2_features/arcing/ already exist.
0
100
用时： 254.92497324943542
5.pkl 2971
../data/2_features/arcing/ already exist.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
用时： 3437.8266990184784
6.pkl 2159
../data/2_features/arcing/ already exist.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
用时： 2543.643310546875
7.pkl 4522
../data/2_features/arcing/ already exist.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
用时： 4943.475087881088
8.pkl 1622
../data/2_features/arcing/ already exist.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
用时： 1813.3839089870453
9.pkl 1393
../data/2_features/arcing/ already exist.
0
100
200
300
400
500
600
700
800
900